In [1]:
# A person class for spoofing Google

# your phone tracks:
# steps
# heartrate
# current location
# activity type: sleep, wake, walking, driving

# it can make inferences about calories burned, 
# do address place lookups, and current establishments

# location is updated > 14x/hour
# ad domains
# camera
# microphone
# call log
# SMS messages

> Since the spoofer is kind of like the Other's "heart" and "eyes" in terms of the data it produces, the idea of it existing in the gallery is kind of spooky, in a good way!  Sort of like physical proof of the coldness and inaccuracy of the data collected.  I like how the first image looks very technical.  The fact that it's generating quantitative versions of "emotions" makes that really funny.  

> I've also been thinking about how this project might exist online as well.  Since it is a simulated person, having feeds for the "eyes" (the Google Earth images", the "heart" (moods/sleep/BPM), and whatever may correlate to the Amazon searches ("brain", maybe?) could be interesting as well!  

In [2]:
import time
import logging
import json
from datetime import datetime
import csv,sys

import simplejson as json
from shapely.geometry import Point
from shapely.wkt import dumps, loads
from peewee import *

In [3]:
DEBUG = False
logger = logging.getLogger(__name__)

if DEBUG:
    db = SqliteDatabase(':memory:')
else:
    db = SqliteDatabase('other.db')

In [4]:
heartratedata = "../data/heartrate.json"
# this is 24 hours of heart-rate sampled on average every 2.76 seconds
# time has the format "19:14:00" "%H:%M:%S"
# appears to be UTC/GMT

# [{
#   "time" : "08:00:07",
#   "value" : {
#     "bpm" : 78,
#     "confidence" : 2
#   }
# }

sleepdata = "../data/sleep.json"


timepointdata = "../data/locations.csv"
# Start Time,End Time,Name,Latitude,Longitude
# lat/lng has been transformed and spoofed for Northern CA
# ten days were compressed into one for variety. 

stepdata = "../data/steps.json"
# datetime has the format "Tue 19:14:00" "%a %H:%M:%S"
# appears to be UTC/GMT

# [{
#   "dateTime" : "Sat 08:03:00",
#   "value" : "0"
# },

In [5]:
class BaseModel(Model):
    class Meta:
        database = db

class Person(BaseModel):
    name = CharField()

    def get_mytimepoints(self):
        return self.timepoints


    def get_myheartbeats(self):
        return self.heartbeats


class Heart(BaseModel):
    actor = ForeignKeyField(Person, backref='heartbeats')
    timestamp = IntegerField()
    bpm = IntegerField()

    
class Brain(BaseModel):
    actor = ForeignKeyField(Person, backref='shoppingurls')
    timestamp = TimestampField()
    urls = []
    
class Eyes(BaseModel):
    actor = ForeignKeyField(Person, backref='streetviews')
    timestamp = TimestampField()

    streetviews = []
    
class Legs(BaseModel):
    actor = ForeignKeyField(Person, backref='steps')
    timestamp = TimestampField()

    steps = []
    
class Timepoint(BaseModel):
    actor = ForeignKeyField(Person, backref='timepoints')
    timestamp = IntegerField()
    point = CharField()
    mode = CharField()

    def __repr__(self):
        return self.timestamp, self.mode, loads(self.point)


class Step(BaseModel):
    # do we count steps individually
    # or count them in a 24 hour period?
    actor = ForeignKeyField(Person, backref='steps')
    steps = IntegerField()
    timestamp = TimestampField()

In [6]:
def gimmeSeconds(thetime="08:02:02", thefmt="%H:%M:%S"):
    s = datetime.strptime(thetime, thefmt).second * 1
    ms = datetime.strptime(thetime, thefmt).minute * 60
    hs = datetime.strptime(thetime, thefmt).hour * 3600
    return int(s+ms+hs)

def gimmecurseconds():
    now = datetime.now()
    secs_since_midnight = (now - now.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    return int(secs_since_midnight)

In [7]:
def gimmeclosestkv(mykeys):
#     mykeys = set().union(*(d.keys() for d in alistofdicts))
    mykey = min(mykeys, key=lambda x:abs(x - gimmecurseconds() ))
    q = Heart.select().where(Heart.timestamp == int(mykey))
    for entry in q:
        mybpm = entry.bpm    

    return mykey,mybpm

def gimmeclosestplace():
#     mykeys = set().union(*(d.keys() for d in alistofdicts))
    # get the keys by querying the timepoints
    mykeys = []
    q = Timepoint.select()
    for entry in q:
        mykeys.append(int(entry.timestamp))

    mykey = min(mykeys, key=lambda x:abs(x - gimmecurseconds() ))

    q = Timepoint.select().where(Timepoint.timestamp == int(mykey))
    for entry in q:
        myplce = entry.point

    return mykey,myplce

In [8]:
# Create the table.
# Run this ONLY ONCE IN PRODUCTION!
db.create_tables([Person])

other = Person.create(name='OTHER')
other.save()

1

In [9]:
# Connect to our database.
# db.connect()

# Create the tables.
# Run this ONLY ONCE IN PRODUCTION!


def createHeartdb():
    # Run this ONCE!
    db.create_tables([Heart])
    
    with open(heartratedata, 'r') as f:
        json_text = f.read()

    heartrate_list = json.loads(json_text)

    for rec in heartrate_list:
        secs = gimmeSeconds(rec['time'])
        bpm = rec['value']['bpm']
        beats = Heart.create(actor=other, timestamp=secs, bpm=bpm)
        beats.save()

createHeartdb()

In [10]:
def createTimepointdb():
    # Run this ONCE!
    db.create_tables([Timepoint])
    
    with open(timepointdata, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        # This skips the first row of the CSV file.
        next(csvreader)
        for row in csvreader:
#             print( gimmeSeconds(thetime=row[0]), float(row[3]), float(row[4]) )
            # Start Time,End Time,Name,Latitude,Longitude
    #         actor = ForeignKeyField(Person, backref='timepoints')
    #         timestamp = TimestampField()
    #         point = CharField()
    #         mode = CharField()
            mypoint = Point(float(row[3]), float(row[4]))
            tp = Timepoint.create(actor=other, timestamp=gimmeSeconds(thetime=row[0]), point=dumps(mypoint), mode="WALK")
            tp.save()

createTimepointdb()

In [11]:
# You don't want to run this on every query
heartrate_keylist = []
q = Heart.select(Heart.timestamp)
for t in q:
    heartrate_keylist.append( int(t.timestamp) )

In [16]:
# what's our heart rate?
print("secs since mid",gimmecurseconds(),"closest key, bpm",gimmeclosestkv(heartrate_keylist))

secs since mid 3958 closest key, bpm (3955, 103)


In [15]:
# what's our location?
print("secs since mid",gimmecurseconds(),"closest key, bpm",gimmeclosestplace())

secs since mid 3944 closest key, bpm (4655, 'POINT (37.8683951496994027 -122.4989054513820008)')
